In [3]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import statsmodels.api as sm
from scipy import stats
import pwlf
from sklearn import mixture
import scipy.linalg as la
from statsmodels.graphics.gofplots import qqplot_2samples
from scipy import stats
from scipy.interpolate import CubicSpline
from localreg import *
import os.path
from os import path
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.colors import Normalize
import matplotlib.lines as mlines
from matplotlib import gridspec
import os.path
from os import path

from datetime import datetime

date = datetime.now().strftime("%Y_%m_%d")

import rpy2
# import rpy2's package module
import rpy2.robjects.packages as rpackages

# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list
# R package names
packnames = ('lokern')

# R vector of strings
from rpy2.robjects.vectors import StrVector

# Selectively install what needs to be install.
# We are fancy, just because we can.
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))


from rpy2.robjects import FloatVector

lokern = rpackages.importr('lokern')


from matplotlib.offsetbox import AnchoredOffsetbox, TextArea, HPacker, VPacker


%matplotlib qt

data_all = []
with open('../data/ModelingResults/raw_cleaned.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in reader:
        data_all.append(row)

headerS = list(np.array(data_all[0])[[0,1,12,14,15,20,17,18,19]])

dataS = np.array(data_all)[1:,[0,1,12,14,15,20,17,18,19]]
for i in range(len(dataS)):
    dataS[i,1] = dataS[i,1][-1]

dataS = dataS.astype(float)

data_allI = []
with open('../data/ModelingResults/all_student_measurements.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in reader:
        data_allI.append(row)
        
header_allI = list(np.array(data_allI[0]))

student = np.array(data_allI[1:])[:,5]

headerI = ['Hour', 'replicate', 'well', 'length', 'width']

names = np.unique(student)

dataI = np.array(data_allI[1:])[:,[2,4,4,7,8]]

for i in range(len(dataI)):
    dataI[i,1] = ord(dataI[i,1][0])-64
    dataI[i,2] = dataI[i,2][1:]

dataI = dataI.astype(float)

header_combined_all = ['replicate','hour','TOF','length','norm.EXT','width','EXT','length x width']
medians_combined_all = np.zeros((6*72,8))
count = 0
for t in range(72):
    SortTime = dataS[dataS[:,0]==t+1]
    ImTime = dataI[dataI[:,0]==t+1]
    for r in range(6):
        SortRep = SortTime[SortTime[:,1]==r+1]
        ImRep = ImTime[ImTime[:,1]==r+1]
        timeofflight = SortRep[:,3]
        extinction = SortRep[:,5]
        length = ImRep[:,3]
        width = ImRep[:,4]
        ext = SortRep[:,4]
        area = np.multiply(length, width)
        medians_combined_all[count] = np.array([r,t,np.median(timeofflight),np.median(length),
                                               np.median(extinction),np.median(width),
                                                np.median(ext), np.median(area)])
        count+=1
medians_combined_all=medians_combined_all[medians_combined_all[:,0]!=0]

header_combined = ['replicate','hour','well','TOF','length','norm.EXT','width', 'EXT']
medians_combined = np.zeros((6*11*72,8))
count = 0
for t in range(72):
    SortTime = dataS[dataS[:,0]==t+1]
    ImTime = dataI[dataI[:,0]==t+1]
    for r in range(6):
        SortRep = SortTime[SortTime[:,1]==r+1]
        ImRep = ImTime[ImTime[:,1]==r+1]
        wellnames = np.unique(ImRep[:,2])
        for w in wellnames:
            SortWell = SortRep[SortRep[:,2]==w]
            ImWell = ImRep[ImRep[:,2]==w]
            timeofflight = SortWell[:,3]
            extinction = SortWell[:,5]
            length = ImWell[:,3]
            width = ImWell[:,4]
            ext = SortWell[:,4]
            medians_combined[count] = np.array([r+1,t+1,w,np.median(timeofflight),np.median(length),
                                               np.median(extinction),np.median(width),np.median(ext)])
            count+=1
medians_combined=medians_combined[medians_combined[:,0]!=0]

TOF_length = np.poly1d(np.polyfit(medians_combined_all[:,2],medians_combined_all[:,3], 1))
nEXT_width = np.poly1d(np.polyfit(medians_combined_all[:,4],medians_combined_all[:,5], 1))
def VolumeS(tof,normEXT):
    return np.multiply(TOF_length(tof),np.square(np.divide(nEXT_width(normEXT),2.)))*np.pi
def VolumeI(length, width):
    return np.multiply(length,np.square(np.divide(width,2.)))*np.pi

placeholder = 0

L1 = [[1,13],[1,13],[1,13],[1,13],[1,13],[1,13]]
L2 = [[16.52,21.93],[16.37,22.39],[16.72,21.98],[16.89,23.22],[17.38,22.36],[17.19,23.40]]
L3 = [[26.75,31.94],[26.93,32.96],[26.94,32.89],[27.23,33.10],[27.58,33.63],[27.72,33.86]]
L4 = [[38.20,44.41],[38.37,43.50],[38.66,45.31],[38.56,43.49],[38.99,45.07],[38.96,44.48]]

stages = [L1,L2,L3,L4]
c_cycle = ['#4477AA', '#117733', '#DDCC77', '#CC6677']

Volume = VolumeS(dataS[:,3],dataS[:,5])*1e-6
Combo = np.divide(dataS[:,-1],VolumeS(dataS[:,3],dataS[:,5])*1e-6)
datalength = TOF_length(dataS[:,3])
datawidth = nEXT_width(dataS[:,5])
datarep = dataS[:,1]

if path.exists("Length_time_fit.npy"):
    runboot = input("You've already run the bootstrap analysis. Do you want to re-run it?\nThis may take a while\n[Y\\n]\n")
    if runboot == "n":
        fit_Ls_boot = np.load('../data/ModelingResults/Length_byStage.npy')
        fit_Ws_boot = np.load('../data/ModelingResults/Width_byStage.npy')
        fit_Ls_boot1 = np.load('../data/ModelingResults/Length_byStage_floor&ceiling.npy')
        fit_Ws_boot1 = np.load('../data/ModelingResults/Width_byStage_floor&ceiling.npy')
        iterations = np.shape(fit_Ls_boot)[1]
        regtime_s = np.load('../data/ModelingResults/LW_regtime_bystage.npy')
        regtime_s1 = np.load('../data/ModelingResults/LW_regtime_bystage_floor&ceiling.npy')
    elif runboot == "Y":
        iterations = int(input("\nHow many bootstrap resamplings do you want?\n"))
        
        fit_Ls_boot = np.empty((6,4,iterations,100))
        fit_Ws_boot = np.empty((6,4,iterations,100))
        reglength = np.empty((6,4, 100))
        regtime_s = np.empty((6,4,100))
        regtime_s1 = np.empty((6,4,100))
        fit_Ls_boot1 = np.empty((6,4,iterations,100))
        fit_Ws_boot1 = np.empty((6,4,iterations,100))
        medrep = medians_combined[:,0]
        medhours = medians_combined[:,1]
        medlength = TOF_length(medians_combined[:,3])
        medwidth = nEXT_width(medians_combined[:,5])

        datarep = dataS[:,1]
        datalength = TOF_length(dataS[:,3])
        datawidth = nEXT_width(dataS[:,5])
        datatime = dataS[:,0]

        for r in range(6):
            Rep = r+1
            for s in range(4):
                stage = stages[s][r]
                hourmin = np.around(stage[0])
                hourmax = np.around(stage[1])
                
                hourmin1 = np.ceil(stage[0])
                hourmax1 = np.floor(stage[1])

                bootlength_all = datalength[np.logical_and(datarep==Rep,np.logical_and(datatime>=hourmin,datatime<=hourmax))]
                bootwidth_all = datawidth[np.logical_and(datarep==Rep,np.logical_and(datatime>=hourmin,datatime<=hourmax))]
                boottime = datatime[np.logical_and(datarep==Rep,np.logical_and(datatime>=hourmin,datatime<=hourmax))]
                regtime_s[r,s] = np.linspace(hourmin,hourmax,100)
                numbersamples = len(bootlength_all)
                
                bootlength_all1 = datalength[np.logical_and(datarep==Rep,np.logical_and(datatime>=hourmin1,datatime<=hourmax1))]
                bootwidth_all1 = datawidth[np.logical_and(datarep==Rep,np.logical_and(datatime>=hourmin1,datatime<=hourmax1))]
                boottime1 = datatime[np.logical_and(datarep==Rep,np.logical_and(datatime>=hourmin1,datatime<=hourmax1))]
                regtime_s1[r,s] = np.linspace(hourmin1,hourmax1,100)
                numbersamples1 = len(bootlength_all1)


                for i in range(iterations):
                    count = np.arange(numbersamples)
                    pick = np.random.choice(count,numbersamples,replace = True)
                    boot_length = bootlength_all[pick]
                    boot_width = bootwidth_all[pick]
                    boot_time = boottime[pick]
                    
                    count1 = np.arange(numbersamples1)
                    pick1 = np.random.choice(count1,numbersamples1,replace = True)
                    boot_length1 = bootlength_all1[pick1]
                    boot_width1 = bootwidth_all1[pick1]
                    boot_time1 = boottime1[pick1]
                    
                    fit_Ls_boot[r,s,i] = lokern.lokerns(FloatVector(boot_time),FloatVector(boot_length),0, 100,
                                                        FloatVector(regtime_s[r,s]),False, 2, True)[3]
                    fit_Ws_boot[r,s,i] = lokern.lokerns(FloatVector(boot_time),FloatVector(boot_width),0, 100,
                                                        FloatVector(regtime_s[r,s]),False, 2, True)[3]
                    
                    fit_Ls_boot1[r,s,i] = lokern.lokerns(FloatVector(boot_time1),FloatVector(boot_length1),0, 100,
                                                        FloatVector(regtime_s1[r,s]),False, 2, True)[3]
                    fit_Ws_boot1[r,s,i] = lokern.lokerns(FloatVector(boot_time1),FloatVector(boot_width1),0, 100,
                                                        FloatVector(regtime_s1[r,s]),False, 2, True)[3]
            print('Finished with Replicate {}: Stretcher Model'.format(r+1))
            
        print('\nSaving Files with Today\'s date so that old ones aren\'t overwritten\nYou MUST change name to load files automatically\n')

        np.save('../data/ModelingResults/Length_byStage_'+date+'.npy',fit_Ls_boot)
        np.save('../data/ModelingResults/Width_byStage_'+date+'.npy',fit_Ws_boot)
        np.save('../data/ModelingResults/LW_regtime_bystage_'+date+'.npy',regtime_s)

        np.save('../data/ModelingResults/Length_byStage_floor&ceiling_'+date+'.npy',fit_Ls_boot1)
        np.save('../data/ModelingResults/Width_byStage_floor&ceiling_'+date+'.npy',fit_Ws_boot1)
        np.save('../data/ModelingResults/LW_regtime_bystage_floor&ceiling_'+date+'.npy',regtime_s1)

    else:
        print('\nThat was not one of the options')
else:
    runboot = input("You have not run the bootstrap analysis. Do you want to run it?\nThis may take a while\n*If you choose not to run the analysis, this script will load the mean and variance information from the paper\n[Y\\n]\n")
    if runboot == "Y":
        iterations = int(input("\nHow many bootstrap resamplings do you want?\n"))
        
        fit_Ls_boot = np.empty((6,4,iterations,100))
        fit_Ws_boot = np.empty((6,4,iterations,100))
        reglength = np.empty((6,4, 100))
        regtime_s = np.empty((6,4,100))
        medrep = medians_combined[:,0]
        medhours = medians_combined[:,1]
        medlength = TOF_length(medians_combined[:,3])
        medwidth = nEXT_width(medians_combined[:,5])

        datarep = dataS[:,1]
        datalength = TOF_length(dataS[:,3])
        datawidth = nEXT_width(dataS[:,5])
        datatime = dataS[:,0]

        for r in range(6):
            Rep = r+1
            for s in range(4):
                stage = stages[s][r]
                hourmin = np.ceil(stage[0])
                hourmax = np.floor(stage[1])
                
                hourmin1 = np.ceil(stage[0])
                hourmax1 = np.floor(stage[1])

                bootlength_all = datalength[np.logical_and(datarep==Rep,np.logical_and(datatime>=hourmin,datatime<=hourmax))]
                bootwidth_all = datawidth[np.logical_and(datarep==Rep,np.logical_and(datatime>=hourmin,datatime<=hourmax))]
                boottime = datatime[np.logical_and(datarep==Rep,np.logical_and(datatime>=hourmin,datatime<=hourmax))]
                regtime_s[r,s] = np.linspace(hourmin,hourmax,100)
                numbersamples = len(bootlength_all)
                
                bootlength_all1 = datalength[np.logical_and(datarep==Rep,np.logical_and(datatime>=hourmin1,datatime<=hourmax1))]
                bootwidth_all1 = datawidth[np.logical_and(datarep==Rep,np.logical_and(datatime>=hourmin1,datatime<=hourmax1))]
                boottime1 = datatime[np.logical_and(datarep==Rep,np.logical_and(datatime>=hourmin1,datatime<=hourmax1))]
                regtime_s1[r,s] = np.linspace(hourmin1,hourmax1,100)
                numbersamples1 = len(bootlength_all1)


                for i in range(iterations):
                    count = np.arange(numbersamples)
                    pick = np.random.choice(count,numbersamples,replace = True)
                    boot_length = bootlength_all[pick]
                    boot_width = bootwidth_all[pick]
                    boot_time = boottime[pick]
                    
                    count1 = np.arange(numbersamples1)
                    pick1 = np.random.choice(count1,numbersamples1,replace = True)
                    boot_length1 = bootlength_all1[pick1]
                    boot_width1 = bootwidth_all1[pick1]
                    boot_time1 = boottime1[pick1]
                    
                    fit_Ls_boot[r,s,i] = lokern.lokerns(FloatVector(boot_time),FloatVector(boot_length),0, 100,
                                                        FloatVector(regtime_s[r,s]),False, 2, True)[3]
                    fit_Ws_boot[r,s,i] = lokern.lokerns(FloatVector(boot_time),FloatVector(boot_width),0, 100,
                                                        FloatVector(regtime_s[r,s]),False, 2, True)[3]
                    
                    fit_Ls_boot1[r,s,i] = lokern.lokerns(FloatVector(boot_time1),FloatVector(boot_length1),0, 100,
                                                        FloatVector(regtime_s1[r,s]),False, 2, True)[3]
                    fit_Ws_boot1[r,s,i] = lokern.lokerns(FloatVector(boot_time1),FloatVector(boot_width1),0, 100,
                                                        FloatVector(regtime_s1[r,s]),False, 2, True)[3]
            print('Finished with Replicate {}: Stretcher Model'.format(r+1))

        np.save('../data/ModelingResults/Length_byStage.npy',fit_Ls_boot)
        np.save('../data/ModelingResults/Width_byStage.npy',fit_Ws_boot)
        np.save('../data/ModelingResults/LW_regtime_byStage.npy',regtime_s)

        np.save('../data/ModelingResults/Length_byStage_floor&ceiling.npy',fit_Ls_boot1)
        np.save('../data/ModelingResults/Width_byStage_floor&ceiling.npy',fit_Ws_boot1)
        np.save('../data/ModelingResults/LW_regtime_bystage_floor&ceiling.npy',regtime_s1)

    elif runboot == "n":
        placeholder = 1    
        dV, dVup, dVdown, R_mean, Rup, Rdown, = np.load("../data/ModelingResults/deriv_array.npy")
        counts,counts1 = np.load('../data/ModelingResults/hist_count.npy')
        xbins,ybins,xbins1,ybins1 = np.load('../data/ModelingResults/histbins.npy')
        
        mean_L, mean_W, regtime_s, mean_dL, mean_dW, mean_L1, mean_W1, regtime_s1, mean_dL1, mean_dW1 = np.load('../data/ModelingResults/LW.npy',allow_pickle=True)
        
        LWcov, LWcov1 = np.load('../data/ModelingResults/LW_cov.npy')
        freq,freqdown,frequp = np.load('../data/ModelingResults/freq_array.npy')
        red,red_std,dFdt,dFdt_std = np.load('../data/ModelingResults/def_data.npy')
        
        SignValue = np.load('../data/ModelingResults/FeedingRegime.npy')
    else:
        print('\nThat was not one of the options')
#####################################################################################################        
figs = input("\nDo you want to plot a figure?\n[Y\\n]\n")
ExRep = int(input("\nWhich Replicate do you want to plot?\n*Paper used Replicate 2\n"))
r = ExRep - 1
while figs == "Y":
    if placeholder == 0:
        whichfig = input('\nWhich figure do you want to plot?\nChoose one\n[6, S9]\n')
        
            
        if whichfig == "6":
            fit_dLs_boot = np.empty((6,4,iterations,100))
            fit_dWs_boot = np.empty((6,4,iterations,100))
            mean_dW = np.empty((6,4,100))
            mean_dL = np.empty((6,4,100))

            for r in range(6):
                for s in range(4):
                    fit_dLs_boot[r,s] = np.gradient(fit_Ls_boot[r,s],regtime_s[r,s],axis = 1)
                    fit_dWs_boot[r,s] = np.gradient(fit_Ws_boot[r,s],regtime_s[r,s],axis = 1)
                    mean_dL[r,s] = np.gradient(np.mean(fit_Ls_boot[r,s],axis = 0),regtime_s[r,s])
                    mean_dW[r,s] = np.gradient(np.mean(fit_Ws_boot[r,s],axis = 0),regtime_s[r,s])

            LWcov = np.empty((2,2,6,4,100))

            for r in range(6):
                for s in range(4):
                    for t in range(100):
                        l = fit_dLs_boot[r,s,:,t]
                        w = fit_dWs_boot[r,s,:,t]
                        lw = np.array([l,w])
                        LWcov[:,:,r,s,t] = np.cov(lw)

            error = np.sqrt((np.divide(LWcov[1,1],np.square(mean_dL))+
                            np.divide(np.square(mean_dW)*LWcov[0,0],np.power(mean_dL,4))-
                            np.divide(2*mean_dW*LWcov[0,1],np.power(mean_dL,3))).astype(float))
            
            r= ExRep-1
            plt.figure(figsize=(13,8))
            Rep = r+1
            gs = gridspec.GridSpec(2,1, height_ratios=[3,1]) 
            stages = [L1,L2,L3,L4]
            plt.subplot(gs[1])
            for s in range(3):
                plt.plot(regtime_s[r,s], np.divide(mean_dW[r,s],mean_dL[r,s]), color = "grey")
                plt.fill_between(regtime_s[r,s],np.divide(mean_dW[r,s],mean_dL[r,s])-error[r,s],
                                 np.divide(mean_dW[r,s],mean_dL[r,s])+error[r,s], color = "grey" ,alpha = 0.4)
            #plt.xlim([270,650])
            #plt.xticks([])
            plt.ylabel('Slope: $\\frac{dW}{dL}$', fontsize = 15)
            #plt.ylim([-0.05,0.5])
            plt.xlabel('Time (Hours)', fontsize = 15)

            plt.subplot(gs[0])
            plt.hist2d(datalength[datarep==Rep],datawidth[datarep==Rep],bins = 200, cmap = 'gray')
            plt.colorbar()

            for s in range(4):
                plt.plot(np.mean(fit_Ls_boot[r,s],axis = 0),
                         np.mean(fit_Ws_boot[r,s],axis = 0)
                         ,color = c_cycle[2],lw = 2)
                counts,xbins,ybins = np.histogram2d((fit_Ls_boot[r,s]).flatten(),(fit_Ws_boot[r,s]).flatten(),
                                                    bins=100,normed=True)
                plt.contourf(counts.transpose(),extent=[xbins[0],xbins[-1],ybins[0],ybins[-1]],
                             colors = c_cycle[2],
                             alpha = 0.4, levels = [0.1*np.max(counts),np.max(counts)])
                plt.contour(counts.transpose(),extent=[xbins[0],xbins[-1],ybins[0],ybins[-1]],
                             colors = c_cycle[2],
                             linewidths = 0.5, levels = [0.1*np.max(counts)])
            plt.ylim([15,50])
            plt.xlim([270,650])
            plt.ylabel('Width ($\mu$m)', fontsize = 15)
            plt.xlabel('Length ($\mu$m)', fontsize = 15)
            plt.suptitle('Fit to Replicate {}'.format(Rep), fontsize = 20)
            plt.show()    
          
        elif whichfig == "S9":
            fit_dLs_boot1 = np.empty((6,4,iterations,100))
            fit_dWs_boot1 = np.empty((6,4,iterations,100))
            mean_dW1 = np.empty((6,4,100))
            mean_dL1 = np.empty((6,4,100))

            for r in range(6):
                for s in range(4):
                    fit_dLs_boot1[r,s] = np.gradient(fit_Ls_boot1[r,s],regtime_s1[r,s],axis = 1)
                    fit_dWs_boot1[r,s] = np.gradient(fit_Ws_boot1[r,s],regtime_s1[r,s],axis = 1)
                    mean_dL1[r,s] = np.gradient(np.mean(fit_Ls_boot1[r,s],axis = 0),regtime_s1[r,s])
                    mean_dW1[r,s] = np.gradient(np.mean(fit_Ws_boot1[r,s],axis = 0),regtime_s1[r,s])

            LWcov1 = np.empty((2,2,6,4,100))

            for r in range(6):
                for s in range(4):
                    for t in range(100):
                        l = fit_dLs_boot1[r,s,:,t]
                        w = fit_dWs_boot1[r,s,:,t]
                        lw = np.array([l,w])
                        LWcov1[:,:,r,s,t] = np.cov(lw)

            error = np.sqrt((np.divide(LWcov1[1,1],np.square(mean_dL1))+
                            np.divide(np.square(mean_dW1)*LWcov1[0,0],np.power(mean_dL1,4))-
                            np.divide(2*mean_dW1*LWcov1[0,1],np.power(mean_dL1,3))).astype(float)).astype(float)
            
            r= ExRep-1
            plt.figure(figsize=(13,8))
            Rep = r+1
            gs = gridspec.GridSpec(2,1, height_ratios=[3,1]) 
            stages = [L1,L2,L3,L4]
            plt.subplot(gs[1])
            for s in range(3):
                plt.plot((regtime_s1[r,s]).astype(float), np.divide(mean_dW1[r,s],mean_dL1[r,s]), "grey")
                plt.fill_between((regtime_s1[r,s]).astype(float),
                                 (np.divide(mean_dW1[r,s],mean_dL1[r,s])-error[r,s]).astype(float),
                                 (np.divide(mean_dW1[r,s],mean_dL1[r,s])+error[r,s]).astype(float),
                                 color = "grey",alpha = 0.4)
            #plt.xlim([270,650])
            #plt.xticks([])
            plt.ylabel('Slope: $\\frac{dW}{dL}$', fontsize = 15)
            #plt.ylim([-0.05,0.5])
            plt.xlabel('Time (Hours)', fontsize = 15)

            plt.subplot(gs[0])
            plt.hist2d(datalength[datarep==Rep],datawidth[datarep==Rep],bins = 200, cmap = 'gray')
            plt.colorbar()

            for s in range(4):
                plt.plot(np.mean(fit_Ls_boot1[r,s],axis = 0),
                         np.mean(fit_Ws_boot1[r,s],axis = 0)
                         ,color = c_cycle[2],lw = 2)
                counts,xbins,ybins = np.histogram2d((fit_Ls_boot1[r,s]).flatten(),(fit_Ws_boot1[r,s]).flatten(),
                                                    bins=100,normed=True)
                plt.contourf(counts.transpose(),extent=[xbins[0],xbins[-1],ybins[0],ybins[-1]],
                             colors = c_cycle[2],
                             alpha = 0.4, levels = [0.1*np.max(counts),np.max(counts)])
                plt.contour(counts.transpose(),extent=[xbins[0],xbins[-1],ybins[0],ybins[-1]],
                             colors = c_cycle[2],
                             linewidths = 0.5, levels = [0.1*np.max(counts)])
            plt.ylim([15,50])
            plt.xlim([270,650])
            plt.ylabel('Width ($\mu$m)', fontsize = 15)
            plt.xlabel('Length ($\mu$m)', fontsize = 15)
            plt.suptitle('Fit to Replicate {}'.format(Rep), fontsize = 20)
            plt.show()
            
        else:
            print("\nThat was not one of the options")    
        figs = input("\nDo you want to plot another figure?\n[Y\\n]\n*Figures will not be visible until you have chosen \"n\"\n")
        
        
    if placeholder == 1:
        whichfig = input('\nWhich figure do you want to plot?\nChoose one\n[6, S9]\n')
        
            
        if whichfig == "6":
            r = ExRep - 1
            error = np.sqrt((np.divide(LWcov[1,1],np.square(mean_dL))+
                            np.divide(np.square(mean_dW)*LWcov[0,0],np.power(mean_dL,4))-
                            np.divide(2*mean_dW*LWcov[0,1],np.power(mean_dL,3))).astype(float))

            _cycle = ['#4477AA', '#117733', '#DDCC77', '#CC6677']
            for r in range(1,2):
                plt.figure(figsize=(13,8))
                Rep = r+1
                gs = gridspec.GridSpec(2,1, height_ratios=[3,1]) 
                stages = [L1,L2,L3,L4]
                plt.subplot(gs[1])
                for s in range(3):
                    plt.plot(regtime_s[r,s], np.divide(mean_dW[r,s],mean_dL[r,s]), "grey")
                    plt.fill_between(regtime_s[r,s].astype(float),
                                     (np.divide(mean_dW[r,s],mean_dL[r,s])-error[r,s]).astype(float),
                                     (np.divide(mean_dW[r,s],mean_dL[r,s])+error[r,s]).astype(float),
                                     color = "grey", alpha = 0.4)
                #plt.xlim([270,650])
                #plt.xticks([])
                plt.ylabel('Slope: $\\frac{dW}{dL}$', fontsize = 15)
                #plt.ylim([-0.05,0.5])
                plt.xlabel('Time (Hours)', fontsize = 15)

                plt.subplot(gs[0])
                plt.hist2d(datalength[datarep==Rep],datawidth[datarep==Rep],bins = 200, cmap = 'gray')
                plt.colorbar()

                for s in range(4):
                    plt.plot(mean_L[r,s],
                             mean_W[r,s]
                             ,color = c_cycle[2],lw = 2)
                    plt.contourf(counts[r,s].transpose(),extent=[xbins[r,s,0],xbins[r,s,-1],
                                                                 ybins[r,s,0],ybins[r,s,-1]],
                                 colors = c_cycle[2],
                                 alpha = 0.4, levels = [0.1*np.max(counts[r,s]),np.max(counts[r,s])])
                    plt.contour(counts[r,s].transpose(),extent=[xbins[r,s,0],xbins[r,s,-1],
                                                           ybins[r,s,0],ybins[r,s,-1]],
                                 colors = c_cycle[2],
                                 linewidths = 0.5, levels = [0.1*np.max(counts[r,s])])
                plt.ylim([15,50])
                plt.xlim([270,650])
                plt.ylabel('Width ($\mu$m)', fontsize = 15)
                plt.xlabel('Length ($\mu$m)', fontsize = 15)
                plt.suptitle('Fit to Replicate {}'.format(Rep), fontsize = 20)
                plt.show()
            
        elif whichfig == "S9":
            r = ExRep - 1
            error = np.sqrt((np.divide(LWcov1[1,1],np.square(mean_dL1))+
                            np.divide(np.square(mean_dW1)*LWcov1[0,0],np.power(mean_dL1,4))-
                            np.divide(2*mean_dW1*LWcov1[0,1],np.power(mean_dL1,3))).astype(float))

            _cycle = ['#4477AA', '#117733', '#DDCC77', '#CC6677']
            plt.figure(figsize=(13,8))
            gs = gridspec.GridSpec(2,1, height_ratios=[3,1]) 
            stages = [L1,L2,L3,L4]
            plt.subplot(gs[1])
            for s in range(3):
                plt.plot(regtime_s1[r,s], np.divide(mean_dW1[r,s],mean_dL1[r,s]), color = c_cycle[2])
                plt.fill_between((regtime_s1[r,s]).astype(float),
                                 (np.divide(mean_dW1[r,s],mean_dL[r,s])-error[r,s]).astype(float),
                                 (np.divide(mean_dW1[r,s],mean_dL1[r,s])+error[r,s]).astype(float),
                                 color = "grey",
                                 alpha = 0.4)
            plt.ylabel('Slope: $\\frac{dW}{dL}$', fontsize = 15)
            plt.xlabel('Time (Hours)', fontsize = 15)

            plt.subplot(gs[0])
            plt.hist2d(datalength[datarep==Rep],datawidth[datarep==Rep],bins = 200, cmap = 'gray')
            plt.colorbar()

            for s in range(4):
                plt.plot(mean_L1[r,s],
                         mean_W1[r,s])
                plt.contourf(counts1[r,s].transpose(),extent=[xbins1[r,s,0],xbins1[r,s,-1],
                                                             ybins1[r,s,0],ybins1[r,s,-1]],
                             colors = c_cycle[2],
                             alpha = 0.4, levels = [0.1*np.max(counts1[r,s]),np.max(counts1[r,s])])
                plt.contour(counts1[r,s].transpose(),extent=[xbins1[r,s,0],xbins1[r,s,-1],
                                                       ybins1[r,s,0],ybins1[r,s,-1]],
                             colors = c_cycle[2],
                             linewidths = 0.5, levels = [0.1*np.max(counts1[r,s])])
            plt.ylim([15,50])
            plt.xlim([270,650])
            plt.ylabel('Width ($\mu$m)', fontsize = 15)
            plt.xlabel('Length ($\mu$m)', fontsize = 15)
            plt.suptitle('Fit to Replicate {}'.format(Rep), fontsize = 20)
            plt.show()
        else:
            print("\nThat was not one of the options")
        figs = input("\nDo you want to plot another figure?\n[Y\\n]\n*Figures will not be visible until you have chosen \"n\"\n")
        

(as ‘lib’ is unspecified)



You've already run the bootstrap analysis. Do you want to re-run it?
This may take a while
[Y\n]
Y

How many bootstrap resamplings do you want?
20
Finished with Replicate 1: Stretcher Model
Finished with Replicate 2: Stretcher Model
Finished with Replicate 3: Stretcher Model
Finished with Replicate 4: Stretcher Model
Finished with Replicate 5: Stretcher Model
Finished with Replicate 6: Stretcher Model

Saving Files with Today's date so that old ones aren't overwritten
You MUST change name to load files automatically


Do you want to plot a figure?
[Y\n]
Y

Which Replicate do you want to plot?
*Paper used Replicate 2
2

Which figure do you want to plot?
Choose one
[6, S9]
6

Do you want to plot another figure?
[Y\n]
*Figures will not be visible until you have chosen "n"
Y

Which figure do you want to plot?
Choose one
[6, S9]
S9

Do you want to plot another figure?
[Y\n]
*Figures will not be visible until you have chosen "n"
n
